In [48]:
import pandas as pd
import numpy as np
import re

# Load the dataset
em_data_filepath = 'C:/Users/HP 820/loan_prediction/ea-ibf-climada/impact_catalog/emdat_public_2022_08_11_query_uid-IKGfZc.xlsx'

# Read the Excel file, skipping the first 6 rows and specifying the sheet name
dfb = pd.read_excel(em_data_filepath, engine='openpyxl', skiprows=6, sheet_name='emdat data')

# Filter for Flood-related disasters
flood_grp = ['Flood']
dfb_fl = dfb[dfb['Disaster Type'].isin(flood_grp)]

# Specify the East African countries of interest
ea_cntr = ['Djibouti', 'Eritrea', 'Ethiopia', 'Kenya', 'Rwanda', 'Somalia', 'South Sudan', 'Sudan (the)', 
           'Tanzania, United Republic of']

# Filter the data for East African countries
dfb_fl_ea = dfb_fl[dfb_fl['Country'].isin(ea_cntr)]

# Extract relevant columns for mapping
dfb_fl_ea_map = dfb_fl_ea[['Dis No', 'Year', 'Country', 'ISO', 'Location']].copy()

# Replace specific patterns in the 'Location' column
dfb_fl_ea_map['loc_str1'] = dfb_fl_ea_map['Location'].str.replace(' and ', ',', regex=False)
dfb_fl_ea_map['loc_str2'] = dfb_fl_ea_map['loc_str1'].str.replace(' & ', ',', regex=False)
dfb_fl_ea_map['loc_str3'] = dfb_fl_ea_map['loc_str2'].str.replace('&', ',', regex=False)
dfb_fl_ea_map['loc_str4'] = dfb_fl_ea_map['loc_str3'].str.replace(';', ',', regex=False)

# Split the modified location string into a list
dfb_fl_ea_map['loc_list'] = dfb_fl_ea_map['loc_str4'].str.split(',')

# Explode the list to create multiple rows for each location
df_fl = dfb_fl_ea_map.explode('loc_list').copy()

# Add a unique row index
df_fl['row_idx'] = ['fl_' + str(idx) for idx in range(len(df_fl))]

# Keep the necessary columns
df1_fl = df_fl[['Dis No', 'Year', 'Country', 'ISO', 'loc_list', 'Location', 'row_idx']].copy()

# Reset index for the exploded DataFrame
df2_fl = df1_fl.reset_index(drop=True)

# Remove leading whitespace from the location list
df2_fl['loc_list_a'] = df2_fl['loc_list'].str.strip()

# Select the final columns for the output
df3_fl = df2_fl[['Dis No', 'Year', 'Country', 'ISO', 'loc_list', 'loc_list_a', 'Location', 'row_idx']]

# Save the processed data to a CSV file
output_filepath = 'C:/Users/HP 820/loan_prediction/ea-ibf-climada/ea_flood_location.csv'
df3_fl.to_csv(output_filepath, index=False)

# Process missing location data (if needed)
# Uncomment the following section if you have missing location data to process

# db = pd.read_csv('/home/bulbul/Documents/07-2022/impact_weather_icpac/lab/ea_ibf_data_resources/impact/em-dat/ea_flood_missing_location.csv')
# db['code'] = db['Dis No'].str.split('-').str[-1]
# db['loc_list'] = db['Location'].str.split(',')
# db1 = db.explode('loc_list')
# db2 = db1.drop_duplicates('code')
# code_list = db2['code'].tolist()
# db1.to_csv('/home/bulbul/Documents/07-2022/impact_weather_icpac/lab/ea_ibf_data_resources/impact/em-dat/ea_flood_missing_location_v1.csv')


c:\Users\HP 820\loan_prediction\ea-ibf-climada\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [42]:
dfb_fl_ea.info()

<class 'pandas.core.frame.DataFrame'>
Index: 269 entries, 33 to 4631
Data columns (total 50 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Dis No                                     269 non-null    object 
 1   Year                                       269 non-null    int64  
 2   Seq                                        269 non-null    int64  
 3   Glide                                      53 non-null     object 
 4   Disaster Group                             269 non-null    object 
 5   Disaster Subgroup                          269 non-null    object 
 6   Disaster Type                              269 non-null    object 
 7   Disaster Subtype                           204 non-null    object 
 8   Disaster Subsubtype                        0 non-null      object 
 9   Event Name                                 0 non-null      object 
 10  Country                      

In [46]:
db=pd.read_csv('C:/Users/HP 820/loan_prediction/202501-impact-events/202501-impact-events/dataframes/ea_flood_missing_location.csv')

db['code']=db['Dis No'].str.split('-').str[-1]

db['loc_list'] = db['Location'].str.split(',')
db1=db.explode('loc_list')
#db2=db1.drop_duplicates('loc_list')

db2=db1.drop_duplicates('code')
code_list=db2['code'].tolist()
#db1.to_csv('/home/bulbul/Documents/07-2022/impact_weather_icpac/lab/ea_ibf_data_resources/impact/em-dat/ea_flood_missing_location_v1.csv')

In [47]:
db2

,Dis No,Location,code,loc_list
0,1993-0005-TZA,"Lushoto, Korogwe districts (Tanga region)",TZA,Lushoto
3,1994-0507-DJI,"Djibouti city, Holhol, Ali Addeh, Tadjoura dis...",DJI,Djibouti city
4,1994-0194-ETH,"North Shoa, South Welo, Tach Gayint district (...",ETH,North Shoa
8,1996-0474-KEN,Nyanza province,KEN,Nyanza province
12,1994-0217-SDN,"El Fasher, Nyala (Darfur state), Hamrat El Wiz...",SDN,El Fasher
13,1994-0573-SOM,South,SOM,South


In [12]:
import geopandas as gp


gaul_adm2=gp.read_file('C:/Users/HP 820/loan_prediction/202501-impact-events/202501-impact-events/shpfiles/ea_adm2_gaul.shp')



In [13]:
gaul_adm2

,gml_id,fid,ADM2_CODE,ADM2_NAME,ADM1_CODE,ADM1_NAME,STATUS,DISP_AREA,ADM0_CODE,ADM0_NAME,geometry
0,None,20,32854,Al Mahabishah,3418,Hajjah,Member State,NO,269,Yemen,"MULTIPOLYGON (((43.45552 15.97692, 43.45554 15..."
1,None,8004,12847,Ouadda,856,Haute-Kotto,Member State,NO,49,Central African Republic,"POLYGON ((23.2875 8.76171, 23.28112 8.75005, 2..."
2,None,8005,12848,Yalinga,856,Haute-Kotto,Member State,NO,49,Central African Republic,"POLYGON ((24.2722 8.27699, 24.2739 8.27343, 24..."
3,None,57,32877,Al Makhadir,3419,Ibb,Member State,NO,269,Yemen,"POLYGON ((44.26976 14.20502, 44.26981 14.20496..."
4,None,59,32853,Al Miftah,3418,Hajjah,Member State,NO,269,Yemen,"POLYGON ((43.54572 15.9774, 43.55705 15.97485,..."
...,...,...,...,...,...,...,...,...,...,...,...
1449,None,33989,32849,Abs,3418,Hajjah,Member State,NO,269,Yemen,"POLYGON ((43.22042 16.18735, 43.22144 16.18402..."
1450,None,34013,32850,Aflah Al Yaman,3418,Hajjah,Member State,NO,269,Yemen,"POLYGON ((43.44169 15.98401, 43.44304 15.98109..."
1451,None,34019,32851,Aflah Ash Shawm,3418,Hajjah,Member State,NO,269,Yemen,"POLYGON ((43.43349 16.07773, 43.43851 16.07462..."
1452,None,34039,32852,Al Jamimah,3418,Hajjah,Member State,NO,269,Yemen,"POLYGON ((43.5759 16.09728, 43.57911 16.09281,..."


In [14]:
import geopandas as gp
import pandas as pd

adm0=gp.read_file('C:/Users/HP 820/loan_prediction/202501-impact-events/202501-impact-events/shpfiles/ea_adm0_geoboundaries.shp')
adm1=gp.read_file('C:/Users/HP 820/loan_prediction/202501-impact-events/202501-impact-events/shpfiles/ea_adm1_geoboundaries.shp')
adm2=gp.read_file('C:/Users/HP 820/loan_prediction/202501-impact-events/202501-impact-events/shpfiles/ea_adm2_geoboundaries.shp')

db=pd.concat([adm0,adm1,adm2])

db1=db.drop_duplicates('shapeID')
db1.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 1155 entries, 0 to 971
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   shapeName   1155 non-null   object  
 1   shapeISO    1151 non-null   object  
 2   shapeID     1155 non-null   object  
 3   shapeGroup  1155 non-null   object  
 4   shapeType   1155 non-null   object  
 5   geometry    1155 non-null   geometry
dtypes: geometry(1), object(5)
memory usage: 63.2+ KB


In [15]:
db1=db[db['shapeGroup']=='SDN']
#db1.shapeName.tolist()
db2=db1.sort_values('shapeName')
db3=db2[['shapeName','shapeType','shapeID']]
pd.set_option('display.max_columns', 4)
pd.set_option('display.max_rows', 200)
print(db3)

                shapeName shapeType              shapeID
522             Abu Hamed      ADM2    SDN-ADM2-3_0_0-B8
544            Abu Houjar      ADM2   SDN-ADM2-3_0_0-B30
654             Abu Jabra      ADM2  SDN-ADM2-3_0_0-B140
576           Abu Jubaiha      ADM2   SDN-ADM2-3_0_0-B62
624           Abu Karinka      ADM2  SDN-ADM2-3_0_0-B110
650             Abu Zabad      ADM2  SDN-ADM2-3_0_0-B136
640        Abyei - Muglad      ADM2  SDN-ADM2-3_0_0-B126
584        Abyei PCA Area      ADM2   SDN-ADM2-3_0_0-B70
655                 Adila      ADM2  SDN-ADM2-3_0_0-B141
567                  Agig      ADM2   SDN-ADM2-3_0_0-B53
635               Ailliet      ADM2  SDN-ADM2-3_0_0-B121
583              Al Buram      ADM2   SDN-ADM2-3_0_0-B69
531               Al Daba      ADM2   SDN-ADM2-3_0_0-B17
680             Al Mafaza      ADM2  SDN-ADM2-3_0_0-B166
644                Al Qoz      ADM2  SDN-ADM2-3_0_0-B130
643              Al Sunut      ADM2  SDN-ADM2-3_0_0-B129
588            Al Tadamon      

In [16]:
dfb_fl_ea['location_count']=dfb_fl_ea.Location.str.count(',')
dfb_fl_ea['adm_lvl_count']=dfb_fl_ea['Adm Level'].str.count(';')
dfb_fl_ea['adm1_count']=dfb_fl_ea['Admin1 Code'].str.count(';')
dfb_fl_ea['adm2_count']=dfb_fl_ea['Admin2 Code'].str.count(';')

C:\Users\HP 820\AppData\Local\Temp\ipykernel_7036\3950927091.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb_fl_ea['location_count']=dfb_fl_ea.Location.str.count(',')
C:\Users\HP 820\AppData\Local\Temp\ipykernel_7036\3950927091.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb_fl_ea['adm_lvl_count']=dfb_fl_ea['Adm Level'].str.count(';')
C:\Users\HP 820\AppData\Local\Temp\ipykernel_7036\3950927091.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


## replacing GAUL Admin1 Code into GEOB admin2 codes

In [17]:
dfb_fl_ea['gaul_adm1_list']=dfb_fl_ea['Admin1 Code'].str.split(';')


em_dat_path='./'


gaul_adm1_dict_db=pd.read_csv(f'{em_dat_path}em_dat_ea_gaul_geob_adm1_codeshare.csv')

gaul_adm1_dict = dict(zip(gaul_adm1_dict_db.gaul_adm1_code, gaul_adm1_dict_db.geob_adm1_shapeID))


dfb_fl_ea_adm1=dfb_fl_ea[dfb_fl_ea['gaul_adm1_list'].notna()]

dfb_fl_ea_adm1['gaul_adm1_list_int']=dfb_fl_ea_adm1['gaul_adm1_list'].apply(lambda x: [int(i) for i in x])


dfb_fl_ea_adm1['gaul_adm1_geob'] = dfb_fl_ea_adm1['gaul_adm1_list_int'].apply(lambda x: [gaul_adm1_dict[i] for i in x])

#dfb_dr_adm1

dfb_fl_ea_adm1_a=dfb_fl_ea_adm1[['Dis No','gaul_adm1_geob']]

dfb_fl_ea_adm1_a.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99 entries, 1033 to 4571
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Dis No          99 non-null     object
 1   gaul_adm1_geob  99 non-null     object
dtypes: object(2)
memory usage: 2.3+ KB


C:\Users\HP 820\AppData\Local\Temp\ipykernel_7036\1139056622.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb_fl_ea['gaul_adm1_list']=dfb_fl_ea['Admin1 Code'].str.split(';')
C:\Users\HP 820\AppData\Local\Temp\ipykernel_7036\1139056622.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb_fl_ea_adm1['gaul_adm1_list_int']=dfb_fl_ea_adm1['gaul_adm1_list'].apply(lambda x: [int(i) for i in x])
C:\Users\HP 820\AppData\Local\Temp\ipykernel_7036\1139056622.py:17: SettingWithCopyWarning: 
A value is trying

 ## replace adm1 code with adm2 sub codes

In [18]:
import geopandas as gp
import numpy as np

# Define the file path to the shapefile
geob_adm1_adm2_filepath = 'C:/Users/HP 820/loan_prediction/202501-impact-events/202501-impact-events/shpfiles/ea_adm2_geoboundaries_adm1_code.shp'

# Read the shapefile
geob_db = gp.read_file(geob_adm1_adm2_filepath)

# Inspect the column names
print(geob_db.columns)

# Select relevant columns (adjust based on actual column names in shapefile)
geob_db1 = geob_db[['adm1_shape', 'shapeID']]  # Replace with actual column names
geob_db2 = geob_db1.groupby('adm1_shape').agg({'shapeID': lambda x: list(x)})
geob_db3 = geob_db2.reset_index()
geob_db3.columns = ['gaul_adm1_geob', 'shapeID']

# Create a dictionary for mapping
geob_db3_dict = dict(zip(geob_db3.gaul_adm1_geob, geob_db3.shapeID))

# Ensure dfb_fl_ea_adm1_a is defined
# Assuming dfb_fl_ea_adm1_a has a column 'gaul_adm1_geob' with values matching the keys in geob_db3_dict
dfb_fl_ea_adm1_a['geob_adm2'] = dfb_fl_ea_adm1_a['gaul_adm1_geob'].apply(lambda x: [geob_db3_dict[i] for i in x if i in geob_db3_dict])

# Define a function to flatten lists of lists
def flatten(l):
    return [item for sublist in l for item in sublist]

# Flatten the geob_adm2 column
aa_list = []
for idx, row in dfb_fl_ea_adm1_a.iterrows():
    aa = flatten(row['geob_adm2'])
    aa_list.append(aa)

dfb_fl_ea_adm1_a['geob_adm2_list'] = aa_list

# Extract the relevant columns
dfb_fl_part1 = dfb_fl_ea_adm1_a[['Dis No', 'geob_adm2_list']]
print(dfb_fl_part1)


c:\Users\HP 820\loan_prediction\ea-ibf-climada\venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: C:/Users/HP 820/loan_prediction/202501-impact-events/202501-impact-events/shpfiles/ea_adm2_geoboundaries_adm1_code.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


Index(['shapeName', 'shapeISO', 'shapeID', 'shapeGroup', 'shapeType',
       'adm1_shape', 'geometry'],
      dtype='object')
             Dis No                                     geob_adm2_list
1033  2000-0259-SOM  [SOM-ADM2-3_0_0-B18, SOM-ADM2-3_0_0-B38, SOM-A...
1034  2000-0778-SOM  [SOM-ADM2-3_0_0-B17, SOM-ADM2-3_0_0-B40, SOM-A...
1046  2000-0790-TZA  [TZA-ADM2-3_0_0-B22, TZA-ADM2-3_0_0-B32, TZA-A...
1169  2001-0464-ETH  [ETH-ADM2-3_0_0-B2, ETH-ADM2-3_0_0-B38, ETH-AD...
1252  2002-0225-ETH  [ETH-ADM2-3_0_0-B70, ETH-ADM2-3_0_0-B71, ETH-A...
1371  2001-0672-RWA  [RWA-ADM2-3_0_0-B18, RWA-ADM2-3_0_0-B19, RWA-A...
1372  2001-0417-SDN  [SDN-ADM2-3_0_0-B23, SDN-ADM2-3_0_0-B24, SDN-A...
1442  2002-0273-RWA  [RWA-ADM2-3_0_0-B1, RWA-ADM2-3_0_0-B2, RWA-ADM...
1503  2002-0515-SDN  [SDN-ADM2-3_0_0-B23, SDN-ADM2-3_0_0-B24, SDN-A...
1509  2002-0715-SOM  [SOM-ADM2-3_0_0-B21, SOM-ADM2-3_0_0-B25, SOM-A...
1645  2003-0204-ETH  [ETH-ADM2-3_0_0-B1, ETH-ADM2-3_0_0-B14, ETH-AD...
1730  2003-0368-SDN  [

C:\Users\HP 820\AppData\Local\Temp\ipykernel_7036\2799980077.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb_fl_ea_adm1_a['geob_adm2'] = dfb_fl_ea_adm1_a['gaul_adm1_geob'].apply(lambda x: [geob_db3_dict[i] for i in x if i in geob_db3_dict])
C:\Users\HP 820\AppData\Local\Temp\ipykernel_7036\2799980077.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb_fl_ea_adm1_a['geob_adm2_list'] = aa_list


## replacing GAUL Admin2 Code into GEOB admin2 codes

In [19]:
dfb_fl_ea['gaul_adm2_list']=dfb_fl_ea['Admin2 Code'].str.split(';')


em_dat_path='./'


gaul_adm2_dict_db=pd.read_excel(f'{em_dat_path}em_dat_ea_gaul_geob_adm2_codeshare_v1.xlsx', engine='openpyxl')

gaul_adm2_dict = dict(zip(gaul_adm2_dict_db.gaul_adm2_code, gaul_adm2_dict_db.geob_adm2_shapeID))

gaul_adm2_dict[48481]='TZA-ADM1-3_0_0-B2'
gaul_adm2_dict[48451]='TZA-ADM1-3_0_0-B8'

#48481:TZA-ADM1-3_0_0-B2
#48451:TZA-ADM1-3_0_0-B8

dfb_fl_adm2=dfb_fl_ea[dfb_fl_ea['gaul_adm2_list'].notna()]

dfb_fl_adm2['gaul_adm2_list_int']=dfb_fl_adm2['gaul_adm2_list'].apply(lambda x: [int(i) for i in x])

#dfb_fl_adm2a=dfb_fl_adm2[dfb_fl_adm2['gaul_adm2_list_int']==48481]


dfb_fl_adm2['gaul_adm2_geob'] = dfb_fl_adm2['gaul_adm2_list_int'].apply(lambda x: [gaul_adm2_dict[i] for i in x])

# _b_dict={1:'TZA-ADM1-3_0_0-B2',2:'TZA-ADM1-3_0_0-B8'}

# dfb_fl_adm2['gaul_adm2_geob'] = dfb_fl_adm2['gaul_adm2_geob'].apply(lambda x: [_b_dict[i] for i in x])
# #dfb_dr_a

dfb_fl_part2=dfb_fl_adm2[['Dis No','gaul_adm2_geob']]

dfb_fl_part2.columns=['Dis No','geob_adm2_list']




dfb_fl_part2

C:\Users\HP 820\AppData\Local\Temp\ipykernel_7036\2734334528.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb_fl_ea['gaul_adm2_list']=dfb_fl_ea['Admin2 Code'].str.split(';')
C:\Users\HP 820\AppData\Local\Temp\ipykernel_7036\2734334528.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb_fl_adm2['gaul_adm2_list_int']=dfb_fl_adm2['gaul_adm2_list'].apply(lambda x: [int(i) for i in x])
C:\Users\HP 820\AppData\Local\Temp\ipykernel_7036\2734334528.py:24: SettingWithCopyWarning: 
A value is trying to be

,Dis No,geob_adm2_list
785,2000-0497-ETH,[ETH-ADM2-3_0_0-B70]
796,2000-0399-ETH,"[ETH-ADM2-3_0_0-B48, ETH-ADM2-3_0_0-B17]"
910,2001-0032-KEN,[KEN-ADM2-3_0_0-B46]
1033,2000-0259-SOM,"[SOM-ADM2-3_0_0-B17, SOM-ADM2-3_0_0-B33, SOM-A..."
1039,2000-0490-SOM,[SOM-ADM2-3_0_0-B68]
1164,2001-0448-ETH,[ETH-ADM2-3_0_0-B70]
1170,2001-0769-ETH,[ETH-ADM2-3_0_0-B67]
1214,2002-0265-KEN,"[KEN-ADM2-3_0_0-B20, KEN-ADM2-3_0_0-B44, KEN-A..."
1270,2002-0711-KEN,[KEN-ADM2-3_0_0-B18]
1271,2002-0689-KEN,"[KEN-ADM2-3_0_0-B25, KEN-ADM2-3_0_0-B58]"


In [20]:
import geopandas as gp
import glob

shpfiles=glob.glob('C:/Users/HP 820/loan_prediction/202501-impact-events/202501-impact-events/shpfiles/geoboundary_code_flood*.shp')

for shpfl in shpfiles:
    db=gp.read_file(shpfl)
    db

In [21]:
db

,shapeName,shapeISO,...,shapeType,geometry
0,Burundi,BDI,...,ADM0,"POLYGON ((30.4119 -2.31239, 30.41202 -2.3126, ..."
1,Djibouti,DJI,...,ADM0,"MULTIPOLYGON (((43.18436 11.72844, 43.18447 11..."
2,Eritrea,ERI,...,ADM0,"MULTIPOLYGON (((42.93967 12.84493, 42.93967 12..."
3,Ethiopia,ETH,...,ADM0,"POLYGON ((37.94631 14.8352, 37.95126 14.83274,..."
4,Kenya,KEN,...,ADM0,"POLYGON ((35.93833 4.61944, 35.94423 4.61914, ..."
...,...,...,...,...,...
967,AGAGO,UG-322,...,ADM2,"POLYGON ((33.55052 3.28146, 33.55067 3.27989, ..."
968,KAGADI,None,...,ADM2,"POLYGON ((31.06062 1.16054, 31.06079 1.1605, 3..."
969,KIBAALE,UG-407,...,ADM2,"POLYGON ((31.12178 1.00273, 31.12179 1.00232, ..."
970,GULU,UG-304,...,ADM2,"POLYGON ((32.51042 3.16134, 32.51034 3.16057, ..."


## replacing manually added GEOB admin 1 and admin2 codes into common admin2 codes

In [22]:
import pandas as pd
import geopandas as gp

# File paths
em_dat_path = r'C:/Users/HP 820/loan_prediction/ea-ibf-climada/data_sources/202501-impact-events/202501-impact-events/dataframes/'
shp_path = r'C:/Users/HP 820/loan_prediction/ea-ibf-climada/data_sources/202501-impact-events/202501-impact-events/shpfiles/'

# Reading data
dfb_fl = pd.read_excel(f'{em_dat_path}ea_flood_missing_location_v2.xlsx', engine='openpyxl')

# Splitting data based on presence of geob values
dfb_fl1 = dfb_fl[dfb_fl['geob'].isna()]
dfb_fl2 = dfb_fl[dfb_fl['geob'].notna()]
dfb_fl2['geob_list'] = dfb_fl2['geob'].str.split(',')

# Reading shapefiles
_ken = gp.read_file(f'{shp_path}geoboundary_code_flood/1998-0443-KEN.shp')
_som94 = gp.read_file(f'{shp_path}geoboundary_code_flood/1994-0573-SOM.shp')
_som97 = gp.read_file(f'{shp_path}geoboundary_code_flood/1997-0106-SOM.shp')

# Extracting shapeIDs from shapefiles
_ken_list = _ken['shapeID'].tolist()
_som94_list = _som94['shapeID'].tolist()
_som97_list = _som97['shapeID'].tolist()

# Updating specific rows in dfb_fl1
dfb_fl1.at[38, 'geob'] = _ken_list
dfb_fl1.at[78, 'geob'] = _som94_list
dfb_fl1.at[81, 'geob'] = _som97_list

# Renaming and merging dataframes
dfb_fl1a = dfb_fl1.rename(columns={"geob": "geob_list"})
dfb_fl_a = pd.concat([dfb_fl2, dfb_fl1a])

# Function to extract ADM1 and ADM2 lists
def get_adm1_list(my_list):
    if not isinstance(my_list, list):  # Ensure my_list is a list
        return [], []
    keys = ['ADM1', 'ADM2']
    aa = [[value for value in my_list if key in value] for key in set(keys)][0]
    bb = [[value for value in my_list if key in value] for key in set(keys)][1]
    return aa, bb

# Extracting ADM1 and ADM2 lists
aa_list = []
bb_list = []
for idx, row in dfb_fl_a.iterrows():
    aa, bb = get_adm1_list(row['geob_list'])
    aa_list.append(aa)
    bb_list.append(bb)

dfb_fl_a['geob_adm2_list'] = bb_list
dfb_fl_a['geob_adm1_list'] = aa_list

# Creating ADM1 dataframe
dfb_fl_adm1 = dfb_fl_a[['Dis No', 'geob_adm1_list']]
dfb_fl_adm1_a = dfb_fl_adm1[dfb_fl_adm1['geob_adm1_list'].map(lambda d: len(d)) > 0]

# Mapping ADM2 lists
geob_db3_dict = {}  # Placeholder for actual dictionary mapping
dfb_fl_adm1_a['geob_adm2_list'] = dfb_fl_adm1_a['geob_adm1_list'].apply(lambda x: [geob_db3_dict.get(i, []) for i in x])

# Flattening nested lists
def flatten(l):
    return [item for sublist in l for item in sublist]

aa_list = []
for idx, row in dfb_fl_adm1_a.iterrows():
    aa = flatten(row['geob_adm2_list'])
    aa_list.append(aa)

dfb_fl_adm1_a['geob_adm2_list_a'] = aa_list

# Merging dataframes
dfb_fl_adm1_b = dfb_fl_adm1_a[['Dis No', 'geob_adm2_list_a']]
dfb_fl2 = pd.merge(dfb_fl_a, dfb_fl_adm1_b, on='Dis No', how='left')

# Combining geob_adm2 lists
dfb_fl2['geob_adm2_list_a'] = dfb_fl2['geob_adm2_list_a'].apply(lambda d: d if isinstance(d, list) else [])
dfb_fl2['geob_adm2_list_a'] = (dfb_fl2.geob_adm2_list + dfb_fl2.geob_adm2_list_a).map(set).map(list)

# Final output
dfb_fl_part3 = dfb_fl2[['Dis No', 'geob_adm2_list_a']]
dfb_fl_part3.columns = ['Dis No', 'geob_adm2_list']

dfb_fl_part3


C:\Users\HP 820\AppData\Local\Temp\ipykernel_7036\1378245918.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb_fl2['geob_list'] = dfb_fl2['geob'].str.split(',')
C:\Users\HP 820\AppData\Local\Temp\ipykernel_7036\1378245918.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb_fl_adm1_a['geob_adm2_list'] = dfb_fl_adm1_a['geob_adm1_list'].apply(lambda x: [geob_db3_dict.get(i, []) for i in x])
C:\Users\HP 820\AppData\Local\Temp\ipykernel_7036\1378245918.py:72: SettingWithCopyWarning: 
A value is tryin

,Dis No,geob_adm2_list
0,1994-0507-DJI,[]
1,1994-0507-DJI,[]
2,1994-0507-DJI,[]
3,1994-0507-DJI,[DJI-ADM1-3_0_0-B4]
4,1994-0507-DJI,[DJI-ADM1-3_0_0-B4]
...,...,...
202,1999-0316-SDN,[]
203,1994-0573-SOM,"[SOM-ADM1-3_0_0-B16, SOM-ADM1-3_0_0-B7, SOM-AD..."
204,1997-0106-SOM,"[SOM-ADM1-3_0_0-B7, SOM-ADM1-3_0_0-B1]"
205,1998-0112-TZA,[]


In [23]:
dfb_parts=pd.concat([dfb_fl_part1,dfb_fl_part2,dfb_fl_part3],axis=0)
dfb_parts1=dfb_parts.drop_duplicates('Dis No')

In [24]:
df=dfb_parts.groupby('Dis No').agg({'geob_adm2_list': 'sum'})

In [25]:
df_a=df[df['geob_adm2_list'].map(lambda d: len(d)) > 0]
adisno=df_a.index.tolist()

In [26]:
#df.to_csv('/home/bulbul/Documents/07-2022/impact_weather_icpac/lab/ea_climada/ea-ibf-climada/impact_catalog/test.csv')

In [27]:
dfb_fl_ea1=dfb_fl_ea[~dfb_fl_ea['Dis No'].isin(adisno)]
df1=dfb_fl_ea1[['Dis No','Location']]
df1
#df1.to_csv('/home/bulbul/Documents/07-2022/impact_weather_icpac/lab/ea_climada/ea-ibf-climada/impact_catalog/fl_part4_missing_location.csv')

,Dis No,Location
33,1993-0503-DJI,Djibouti City
41,1990-0582-ETH,Gambela region
44,1990-0352-KEN,NaN
125,1993-0519-ETH,Near Addis Abeba
126,1993-0520-ETH,"Gollam, Gonder regions"
155,1990-0016-TZA,"Lindi, Mtwara, Masasi"
158,1990-0018-TZA,"Moshi, Rombo districts (Kilimanjaro, Rukwa reg..."
209,1993-0143-SDN,Port Sudan
212,1993-0005-TZA,"Lushoto, Korogwe districts (Tanga region)"
213,1993-0157-TZA,Korogwe District


# part4 missing admins
1990-0352-KEN - http://erepository.uonbi.ac.ke/bitstream/handle/11295/66135/Alfred%20Opere.pdf?sequence
    
1996-0473-SOM - https://reliefweb.int/report/somalia/fews-bulletin-july-24-1996
    
2018-0477-RWA - https://reliefweb.int/disaster/fl-2018-000029-rwa
    
2021-0343-ETH - https://floodlist.com/tag/ethiopia

Flood catalogs for EA from 1985

https://drought.unl.edu/archive/Documents/NDMC/Workshops/539/Pres/GHA2_Policelli-floods-1.pdf

In [28]:
import geopandas as gp

_ken=gp.read_file('C:/Users/HP 820/loan_prediction/ea-ibf-climada/data_sources/202501-impact-events/202501-impact-events/shpfiles/geoboundary_code_flood/1990-0352-KEN.shp')

print(_ken['shapeID'].tolist())

['KEN-ADM2-3_0_0-B3', 'KEN-ADM2-3_0_0-B4', 'KEN-ADM2-3_0_0-B6', 'KEN-ADM2-3_0_0-B7', 'KEN-ADM2-3_0_0-B8', 'KEN-ADM2-3_0_0-B11', 'KEN-ADM2-3_0_0-B12', 'KEN-ADM2-3_0_0-B15', 'KEN-ADM2-3_0_0-B21', 'KEN-ADM2-3_0_0-B24', 'KEN-ADM2-3_0_0-B40', 'KEN-ADM2-3_0_0-B53', 'KEN-ADM2-3_0_0-B55', 'KEN-ADM2-3_0_0-B56', 'KEN-ADM2-3_0_0-B59', 'KEN-ADM2-3_0_0-B62', 'KEN-ADM2-3_0_0-B66', 'KEN-ADM2-3_0_0-B70']


In [29]:
_som=gp.read_file('C:/Users/HP 820/loan_prediction/ea-ibf-climada/data_sources/202501-impact-events/202501-impact-events/shpfiles/geoboundary_code_flood/1996-0473-SOM.shp')

print(_som['shapeID'].tolist())

['SOM-ADM2-3_0_0-B5', 'SOM-ADM2-3_0_0-B9', 'SOM-ADM2-3_0_0-B10', 'SOM-ADM2-3_0_0-B13', 'SOM-ADM2-3_0_0-B14', 'SOM-ADM2-3_0_0-B15', 'SOM-ADM2-3_0_0-B16', 'SOM-ADM2-3_0_0-B26', 'SOM-ADM2-3_0_0-B30', 'SOM-ADM2-3_0_0-B33', 'SOM-ADM2-3_0_0-B38', 'SOM-ADM2-3_0_0-B41', 'SOM-ADM2-3_0_0-B43', 'SOM-ADM2-3_0_0-B45', 'SOM-ADM2-3_0_0-B50', 'SOM-ADM2-3_0_0-B53', 'SOM-ADM2-3_0_0-B61', 'SOM-ADM2-3_0_0-B62', 'SOM-ADM2-3_0_0-B65', 'SOM-ADM2-3_0_0-B69']


In [30]:
_ken1=gp.read_file('C:/Users/HP 820/loan_prediction/ea-ibf-climada/data_sources/202501-impact-events/202501-impact-events/shpfiles/geoboundary_code_flood/1996-0474-KEN.shp')

print(_ken1['shapeID'].tolist())

['KEN-ADM2-3_0_0-B8', 'KEN-ADM2-3_0_0-B11', 'KEN-ADM2-3_0_0-B12', 'KEN-ADM2-3_0_0-B24', 'KEN-ADM2-3_0_0-B40', 'KEN-ADM2-3_0_0-B49', 'KEN-ADM2-3_0_0-B53', 'KEN-ADM2-3_0_0-B56', 'KEN-ADM2-3_0_0-B62']


In [35]:
_dji=gp.read_file(r'C:\Users\HP 820\loan_prediction\ea-ibf-climada\data_sources\202501-impact-events\202501-impact-events\shpfiles\geoboundary_code_flood\1996-9119-DJI.shp')

print (_dji['shapeID'].tolist())

['DJI-ADM1-3_0_0-B1', 'DJI-ADM1-3_0_0-B2', 'DJI-ADM1-3_0_0-B3', 'DJI-ADM1-3_0_0-B4', 'DJI-ADM1-3_0_0-B5', 'DJI-ADM1-3_0_0-B6']


## part4 repalce 

In [37]:
import pandas as pd
import geopandas as gp

# Adjusted path for the shapefile
geob_db = gp.read_file(r'C:\Users\HP 820\loan_prediction\ea-ibf-climada\data_sources\202501-impact-events\202501-impact-events\shpfiles\ea_adm2_geoboundaries_adm1_code.shp')

# Process the shapefile data
geob_db1 = geob_db[['adm1_shape', 'shapeID']]
geob_db2 = geob_db1.groupby('adm1_shape').agg({'shapeID': lambda x: list(x)})
geob_db3 = geob_db2.reset_index()
geob_db3.columns = ['gaul_adm1_geob', 'shapeID']
geob_db3_dict = dict(zip(geob_db3.gaul_adm1_geob, geob_db3.shapeID))

# Load the CSV file
dfb_fl = pd.read_csv('C:/Users/HP 820/loan_prediction/ea-ibf-climada/data_sources/202501-impact-events/202501-impact-events/dataframes/fl_part4_missing_location_v1.csv')

# Process the DataFrame
dfb_fl1 = dfb_fl[dfb_fl['geob'].notna()]
dfb_fl1['geob_list'] = dfb_fl1['geob'].str.split(',').replace(' ', '')

# Function to get ADM1 and ADM2 lists
def get_adm1_list(my_list):
    keys = ['ADM1', 'ADM2']
    aa = [[value for value in my_list if key in value] for key in set(keys)][0]
    bb = [[value for value in my_list if key in value] for key in set(keys)][1]
    return aa, bb

# Iterate over rows to generate lists
aa_list = []
bb_list = []
for idx, row in dfb_fl1.iterrows():
    aa, bb = get_adm1_list(row['geob_list'])
    aa_list.append(aa)
    bb_list.append(bb)

dfb_fl1['geob_adm2_list'] = bb_list
dfb_fl1['geob_adm1_list'] = aa_list

# Merge and process ADM1 and ADM2 data
dfb_fl_adm1 = dfb_fl1[['Dis No', 'geob_adm1_list']]
dfb_fl_adm1_a = dfb_fl_adm1[dfb_fl_adm1['geob_adm1_list'].map(lambda d: len(d)) > 0]

# Modified part to handle the KeyError
def safe_get_adm2_list(adm1_list):
    return [geob_db3_dict[i] for i in adm1_list if i in geob_db3_dict]

dfb_fl_adm1_a['geob_adm2_list'] = dfb_fl_adm1_a['geob_adm1_list'].apply(safe_get_adm2_list)

# Flatten nested lists
def flatten(l):
    return [item for sublist in l for item in sublist]

aa_list = []
for idx, row in dfb_fl_adm1_a.iterrows():
    aa = flatten(row['geob_adm2_list'])
    aa_list.append(aa)

dfb_fl_adm1_a['geob_adm2_list_a'] = aa_list

# Merge the data back and finalize
dfb_fl_adm1_b = dfb_fl_adm1_a[['Dis No', 'geob_adm2_list_a']]
dfb_fl2 = pd.merge(dfb_fl1, dfb_fl_adm1_b, on='Dis No', how='left')

dfb_fl2['geob_adm2_list_a'] = dfb_fl2['geob_adm2_list_a'].apply(lambda d: d if isinstance(d, list) else [])
dfb_fl2['geob_adm2_list_a'] = (dfb_fl2.geob_adm2_list + dfb_fl2.geob_adm2_list_a).map(set).map(list)

dfb_fl_part4 = dfb_fl2[['Dis No', 'geob_adm2_list_a']]
dfb_fl_part4.columns = ['Dis No', 'geob_adm2_list']


c:\Users\HP 820\loan_prediction\ea-ibf-climada\venv\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: C:\Users\HP 820\loan_prediction\ea-ibf-climada\data_sources\202501-impact-events\202501-impact-events\shpfiles\ea_adm2_geoboundaries_adm1_code.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(
C:\Users\HP 820\AppData\Local\Temp\ipykernel_7036\2045634041.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb_fl_adm1_a['geob_adm2_list'] = dfb_fl_adm1_a['geob_adm1_list'].apply(safe_get_adm2_list)
C:\Users\HP 820\AppData\Local\Temp\ipykernel_7036\2045634041.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [38]:
dfb_fl_part4

,Dis No,geob_adm2_list
0,1993-0503-DJI,[]
1,1990-0582-ETH,[ETH-ADM1-3_0_0-B6]
2,1990-0352-KEN,[]
3,1993-0519-ETH,[ETH-ADM1-3_0_0-B1]
4,1993-0520-ETH,[]
5,1990-0016-TZA,"[TZA-ADM1-3_0_0-B8, TZA-ADM1-3_0_0-B11]"
6,1990-0018-TZA,[]
7,1993-0143-SDN,[]
8,1993-0005-TZA,[]
9,1993-0157-TZA,[]


In [39]:
dfb_parts=pd.concat([dfb_fl_part1,dfb_fl_part2,dfb_fl_part3,dfb_fl_part4],axis=0)
#dfb_parts1=dfb_parts.drop_duplicates('Dis No')
df=dfb_parts.groupby('Dis No').agg({'geob_adm2_list': 'sum'})
df['len_of_list'] = df.geob_adm2_list.agg([len])
df['geob_adm2_list1'] = df['geob_adm2_list'].apply(lambda x: list(set(x)))
#df['len_of_list1'] = df.geob_un.agg([len])
df1=df[['geob_adm2_list1']]
df2=df1.reset_index()
df2.columns=['Dis No','geob_adm2_list']
df2.to_excel("em_dat_ea_flood_adm2_v0.xlsx",index=False) 

C:\Users\HP 820\AppData\Local\Temp\ipykernel_7036\748612427.py:4: FutureWarning: using <built-in function len> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  df['len_of_list'] = df.geob_adm2_list.agg([len])


In [40]:
df2

,Dis No,geob_adm2_list
0,1990-0016-TZA,"[TZA-ADM1-3_0_0-B8, TZA-ADM1-3_0_0-B11]"
1,1990-0018-TZA,[]
2,1990-0352-KEN,[]
3,1990-0582-ETH,[ETH-ADM1-3_0_0-B6]
4,1993-0005-TZA,[]
...,...,...
264,2022-0055-RWA,"[RWA-ADM2-3_0_0-B10, RWA-ADM2-3_0_0-B7, RWA-AD..."
265,2022-0232-RWA,"[RWA-ADM2-3_0_0-B26, RWA-ADM2-3_0_0-B2, RWA-AD..."
266,2022-0258-TZA,[TZA-ADM1-3_0_0-B6]
267,2022-0332-SOM,[]
